In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import transformers

In [4]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
tokenizer.tokenize('안녕 내 이름은 김영관이야')

['안녕', '내', '이름', '##은', '김영', '##관', '##이', '##야']

In [5]:
tokenizer('안녕하세요')

{'input_ids': [2, 11655, 4279, 8553, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [11]:
tokenizer.tokenize('빩뷁녋, 월요일부터 공부를 하네요')
# 정상적이지 않은 단어는 UNK로 표기


['[UNK]', ',', '월요일', '##부터', '공부', '##를', '하네', '##요']

In [12]:
tokenizer.vocab_size
# 저장된 단어의 수

35000

In [14]:
tokenizer.convert_tokens_to_ids(['[UNK]', ',', '월요일', '##부터', '공부', '##를', '하네', '##요'])

[1, 16, 10428, 6406, 7079, 4110, 21347, 4150]

In [15]:
tokenizer.convert_ids_to_tokens([1, 16, 10428, 6406, 7079, 4110, 21347, 41500])
# id를 다시 단어로 표기

['[UNK]', ',', '월요일', '##부터', '공부', '##를', '하네', '[UNK]']

### 토크나이저 훈련

1. 어떤 기법을 사용할지
2. vocab_size 결정 >> 작으면 메모리 이득, 크면 의미를 잘 표현할 수 있다.
3. corpus >> 통계적으로 분석, vocab

##### tokenizer vocab에 수동으로 추가하는 방법도 가능

In [18]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [19]:
tokenizer.tokenize('안녕하세요')
# 영어처리 모델이라 잘 되지 않는다

['ᄋ',
 '##ᅡ',
 '##ᆫ',
 '##ᄂ',
 '##ᅧ',
 '##ᆼ',
 '##ᄒ',
 '##ᅡ',
 '##ᄉ',
 '##ᅦ',
 '##ᄋ',
 '##ᅭ']

In [20]:
multi_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [21]:
multi_tokenizer.tokenize('안녕하세요?')
# multi 모델에는 '안녕'이라는 단어가 vocab에 포함되어 있지 않음

['안', '##녕', '##하', '##세', '##요', '?']

In [22]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [23]:
tokenizer.convert_ids_to_tokens([111,22,33])

['Ġ-', 'Ġ"', 'Ġhave']

In [25]:
tokenizer.convert_tokens_to_ids('grave')

28348

### Bert모델

input = '--- [MASK] ---'

>> 모델 적용 >>

output = '마스킹하기 전의 원래 문장'

**3가지 종류 언어모델**
>> 언어에 대한 통계적 이해를 가진 모델

1. Decoder-only 모델 Auto regressive model (GPT1,GPT2,GPT3)
    - vectorize 된 input을 새로운 text로 decode한다

    - few shot learning 사용 (힌트의 개수에 따라 one,few,..)
    - 단점: 모델이 너무 큼

2. Encoder-only 모델 Auto encoder (Bert,RoBERTa,Electra,albert,...)
    - input을 vectorize 시킨다
    - 자연어처리에서 의미는 vector와 관련이 있다

    - pre-training (비지도학습) >> fine-tuning (추가 훈련, 지도학습)
    - 라벨링이 되지 않은 데이터를 사전학습 -> 추가 훈련
    - Downstream task (번역,문장 분류,질의응답,...)
    - 이유?
        1. 성능이 기존 방식보다 훨씬 좋다
        2. 소량의 데이터로도 학습이 훨씬 빨리 된다
    - CLS 토큰을 문장앞에 넣어 문장을 대표하는 벡터임을 알려줌
    - 각 단어를 나타내는 토큰의 평균을 문장을 보여주는 형식으로 할 수도 있음, 성능 좋은 쪽을 선택
    

3. Encoder-Decoder 모델 (BART,T5,...)

    - Transformer 모델과 구조는 동일하다
    - Pre-training >> fine tuning
    - 생성관련 Task 사용
    

nlp
훈련
배치 사이즈 클수록 성능이 좋아지는 경향이 있음 (gpu 성능이 중요한 이유중 하나)

KOnlp pretained model

1. koelectra
2. klue-roberta
3. koBERT


embedding?
고차원의 자료를 저차원의 자료로 변경하면서 필요한 정보를 보존하는 것을 embedding이라고 함

attention?
Decoder에서 추론 시점에 Encoder 토큰을 얼마나 참조하냐?

self-attention?
Encoder에서 자기 자신에게도 가중치를 부여

BERT 모델 <- Transformer 모델에서 decoder을 제거한 모델

```
질의 응답

[CLS] 포메라니안은 어떤 종이야? [SEP] 포메라니안은 독일 태생의 사냥견으로, 노는 것을 좋아한다. [SEP]

정답: 노는 것을
```


    -Encoder 모델은 생성,task 불가능, 추출은 가능
    -Decoder 모델은 가능

    - chatbot을 Encoder모델로 생성하려면?
    >> (답변1,2,3,4)
    질문 > 뽑아낸다

    - chatbot을 Decoder모델로 생성하려면?
    >> 좀 더 사람같은 느낌, 위험성, 엉뚱한 대답이 나올 때가 있음


Text to Image 모델

- BERT >> 문장을 벡터로 만들어준다

- 벡터를 이미지로 만든다